## Imports

In [1]:
import sys
import os
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import OrderedDict 
from sklearn import metrics, model_selection
from torch.autograd import Function
from torch.optim import Adam

filepath = os.path.abspath('')
sys.path.append(os.path.join(filepath, "..", "compiling_nn"))
sys.path.append(os.path.join(filepath, "..", "datasets", "loan"))
from build_odd import compile_nn
from loan import get_loan_dataset

pd.options.mode.copy_on_write = True

## Load data

In [2]:
np_x, np_y = get_loan_dataset(balancing=True, discretizing=False, hot_encoding=True, rmv_pct=0.985)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([280, 14])


## Metrics, Activations, Loss and Networks definitions

In [3]:
def cm(y_true, y_pred):
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix, display_labels=[False, True])
    return cm_display

def plot_cm(y_true, y_pred):
    cm_display = cm(y_true, y_pred)
    fig, ax = plt.subplots(1, 1, figsize=(4,8))
    cm_display.plot(ax=ax, colorbar=False)

def plot_combine_cm(cms, titles=None):
    n = len(cms)
    fig, axs = plt.subplots(1, n, figsize=(4*n, 8))
    if titles:
        for ax, cm, title in zip(axs, cms, titles):
            cm.plot(ax=ax, colorbar=False)
            ax.set_title(title)
    else:
        for ax, cm in zip(axs, cms):
            cm.plot(ax=ax, colorbar=False)
    fig.tight_layout()

def cov_score(y_true, y_pred):
    labels = np.unique(y_true)
    scores = {}

    for label in labels:
        indices_true = np.where(y_true == label)[0]
        indices_pred = np.where(y_pred == label)[0]
        scores[label] = len(np.intersect1d(indices_true, indices_pred))/len(indices_true)

    return scores

def cross_valid(X, Y, train_func, skf, **kw_train):
    for train_index, test_index in skf.split(X, Y):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        model, y_pred = train_func(x_train, y_train, **kw_train)
        model.eval()
        yield y_pred.detach().round(), y_train, model(x_test).detach(), y_test

def tnot(a): return torch.logical_not(a)
def tor(a,b): return torch.logical_or(a,b)
def tand(a,b): return torch.logical_and(a,b)
def txor(a,b): return torch.logical_xor(a,b)

In [4]:
class StepFunction(Function):
    @staticmethod
    def forward(ctx, input):
        output = torch.where(input>=0, torch.tensor(1.0), torch.tensor(0.0))
        ctx.save_for_backward(input)
        return output
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = torch.zeros_like(input)
        return grad_input
    
class StepActivation(nn.Module):
    def forward(self, input):
        if self.training:
            return torch.sigmoid(input)
        else:
            return StepFunction.apply(input)
    
class AsymMSELoss(nn.Module): # https://www.desmos.com/calculator/zmxcluqhkt
    def __init__(self, p=2):
        super(AsymMSELoss, self).__init__()
        self.p = p

    def forward(self, input, label):
        dif = label - input
        a = torch.square(dif)
        b = a*self.p
        loss = torch.where(dif < 0, b, a)
        loss = torch.mean(loss)
        return loss
    
class AsymBCELoss(nn.Module):
    def __init__(self, p=2):
        super(AsymBCELoss, self).__init__()
        assert(p>0)
        self.p1 = p if p >= 1 else 1
        self.p2 = 1/p if p < 1 else 1

    def forward(self, input, label):
        loss1 = torch.log(1-input+1e-8)
        loss1 = loss1.clamp(min=-100)
        loss2 = torch.log(input+1e-8)
        loss2 = loss2.clamp(min=-100)
        loss = -(self.p1*(1-label)*loss1+self.p2*label*loss2)
        
        loss = torch.mean(loss)
        return loss

In [5]:
class ApproxNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        hl1 = 10

        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,1)),
            ('a2', StepActivation())
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x

class CentralNet(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(
            nn.Linear(input_size,hl1),
            nn.Sigmoid(),
            nn.Linear(hl1,hl2),
            nn.Sigmoid(),
            nn.Linear(hl2,1),
            StepActivation(),
        )
    
    def forward(self, x):
        x = self.nn(x)

        return x

class BoundNetResults():
    def __init__(self, x, xhi, xnn, xlo):
        self.x = x
        self.hi = xhi
        self.nn = xnn
        self.lo = xlo

    def __getattr__(self, name):
        if hasattr(self.x, name):
            return getattr(self.x, name)
        else:
            raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")
    
    def __dir__(self):
        return dir(self.x)
    
    def detach(self):
        self.x = self.x.detach()
        self.hi = self.hi.detach()
        self.nn = self.nn.detach()
        self.lo = self.lo.detach()
        return self
    
    def round(self, *args):
        self.x = self.x.round(*args)
        self.hi = self.hi.round(*args)
        self.nn = self.nn.round(*args)
        self.lo = self.lo.round(*args)
        return self

    def __str__(self):
        return '\n'.join([str(t) for t in (self.x, self.hi, self.nn, self.lo)])

class BoundNetLoss():
    def __init__(self, hi, nn, lo):
        self.hi_loss_fn = hi
        self.nn_loss_fn = nn
        self.lo_loss_fn = lo

    def __call__(self, pred, true):
        self.nn_loss = self.nn_loss_fn(pred.nn, true)

        target = pred.nn.detach()
        self.hi_loss = self.hi_loss_fn(pred.hi, target)
        self.lo_loss = self.lo_loss_fn(pred.lo, target)

        return self

    def backward(self):
        self.nn_loss.backward()
        self.hi_loss.backward()
        self.lo_loss.backward()

class BoundNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # high approx nn (bigger = very long to compute into ODD)
        self.hi = ApproxNet()

        # low approx nn
        self.lo = ApproxNet()

        # nn to approximate (can make it bigger easily)
        self.nn = CentralNet()

    def forward(self, x):
        xhi = self.hi(x)
        xnn = self.nn(x)
        xlo = self.lo(x)
        x = torch.where(xhi>0.5, xhi, torch.where(xlo<0.5, xlo, xnn))
        
        x = BoundNetResults(x, xhi, xnn, xlo)

        return x

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self.nn = nn.Sequential(
            nn.Linear(input_size,hl1),
            nn.Sigmoid(),
            nn.Linear(hl1,hl2),
            nn.Sigmoid(),
            nn.Linear(hl2,1),
            StepActivation(),
        )

    def forward(self, x):
        x = self.nn(x)
        
        return x

## Training functions

In [6]:
def train_model(x, y, model, loss_fn, optimizer, max_epoch):
    for _ in range(max_epoch):
        model.train()
        y_pred = model(x)
        
        loss = loss_fn(y_pred, y)

        model.zero_grad()
        loss.backward()
        optimizer.step()

    return model, y_pred

def train_boundnet(x, y, max_epoch=5000, learning_rate=1e-2, weight_decay=1e-6):
    model = BoundNet()
    loss_fn = BoundNetLoss(AsymBCELoss(50), nn.BCELoss(), AsymBCELoss(.02))
    optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    return train_model(x, y, model, loss_fn, optimizer, max_epoch=max_epoch)

def train_simplenet(x, y, max_epoch=5000, learning_rate=1e-2, weight_decay=1e-6):
    model = SimpleNet()
    loss_fn = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    return train_model(x, y, model, loss_fn, optimizer, max_epoch=max_epoch)

## Network evaluation

In [7]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

def show_activation(act, output):
    # Mean activation per output
    act_ones  = torch.where(output==1, act, torch.zeros(act.size()))
    act_zeros = torch.where(output==0, act, torch.zeros(act.size()))

    mean_ones  = torch.mean(act_ones, dim=0)
    mean_zeros = torch.mean(act_zeros, dim=0)

    # Figure initialization
    fig, ax = plt.subplots(2, 1)
    tick_kw = {'left': False, 'bottom': False, 'labelleft': False}

    # Normalize cmap accross both images
    min_act = min(mean_ones.min().item(), mean_zeros.min().item())
    max_act = max(mean_ones.max().item(), mean_zeros.max().item())

    color_map = 'PRGn'

    ax[0].imshow(mean_zeros.unsqueeze(0), cmap=color_map, vmin=min_act, vmax=max_act)
    ax[0].tick_params(**tick_kw)
    ax[0].set_title("activation moyenne de la couche cachée avec 0 en sortie")

    ax[1].imshow(mean_ones.unsqueeze(0), cmap=color_map, vmin=min_act, vmax=max_act)
    ax[1].tick_params(**tick_kw)
    ax[1].set_title("activation moyenne de la couche cachée avec 1 en sortie")

    # Show text on cells
    for i, (v0, v1) in enumerate(zip(mean_zeros, mean_ones)):
        ax[0].text(i, 0, f"{v0.item():.2f}", ha="center", va="center")
        ax[1].text(i, 0, f"{v1.item():.2f}", ha="center", va="center")
    
    fig.tight_layout()
    plt.show()

def compute_activation(net, layer, data):
    net.eval()
    getattr(net, layer).register_forward_hook(get_activation('__net__'))
    output = net(data).detach()
    act = activation.pop('__net__').squeeze()
    show_activation(act, output)

In [8]:
skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=104)
bnet_split_res = cross_valid(x_data, y_data, train_boundnet, skf)
snet_split_res = cross_valid(x_data, y_data, train_simplenet, skf)

dict_metrics = {(modelname, metric, key): [] for modelname in ("boundnet", "simplenet", "hinet", "lonet") 
                for metric in ("f1score", "coverage0", "coverage1") for key in ("valid", "train")}

for i, ((bt, tt, bv, tv), (st, _, sv, _)) in enumerate(zip(bnet_split_res, snet_split_res)):
    prompts = []
    sep_model = f"{'|':^9}"
    for k, b, s, t in [["valid", bv, sv, tv], ["train", bt, st, tt]]:
        b_f1_score = metrics.f1_score(t, b, average="binary")
        s_f1_score = metrics.f1_score(t, s, average="binary")
        prompts.append(f"{'BoundNet':<15}{b_f1_score:.3f}{sep_model}{'SimpleNet':<15}{s_f1_score:.3f}")
        dict_metrics[('boundnet', 'f1score', k)].append(b_f1_score)
        dict_metrics[('simplenet', 'f1score', k)].append(s_f1_score)

        hi_f1_score = metrics.f1_score(t, b.hi, average="binary")
        lo_f1_score = metrics.f1_score(t, b.lo, average="binary")
        prompts.append(f"{'High':<15}{hi_f1_score:.3f}{sep_model}{'Low':<15}{lo_f1_score:.3f}")
        dict_metrics[('hinet', 'f1score', k)].append(hi_f1_score)
        dict_metrics[('lonet', 'f1score', k)].append(lo_f1_score)

        hi_cov_score = cov_score(t, b.hi)
        lo_cov_score = cov_score(t, b.lo)
        prompts.append(f"{'High':<15}{hi_cov_score[1]:.3f}{sep_model}{'Low':<15}{lo_cov_score[1]:.3f}")
        prompts.append(f"{'High':<15}{hi_cov_score[0]:.3f}{sep_model}{'Low':<15}{lo_cov_score[0]:.3f}")
        dict_metrics[('hinet', 'coverage1', k)].append(hi_cov_score[1])
        dict_metrics[('lonet', 'coverage1', k)].append(lo_cov_score[1])
        dict_metrics[('hinet', 'coverage0', k)].append(hi_cov_score[0])
        dict_metrics[('lonet', 'coverage0', k)].append(lo_cov_score[0])

    sep_tv = f"{'||':^10}"
    print(f"Fold {i+1:3} :            {'Valid':^49}{sep_tv}{'Train':^49}",
          f"\tF1 score      {prompts[0]}{sep_tv}{prompts[4]}",
          f"\t              {prompts[1]}{sep_tv}{prompts[5]}",
          f"\tCoverage      {prompts[2]}{sep_tv}{prompts[6]}",
          f"\t              {prompts[3]}{sep_tv}{prompts[7]}",
          sep='\n')

Fold   1 :                                  Valid                          ||                          Train                      
	F1 score      BoundNet       0.813    |    SimpleNet      0.800    ||    BoundNet       1.000    |    SimpleNet      1.000
	              High           0.769    |    Low            0.765    ||    High           0.817    |    Low            0.851
	Coverage      High           0.714    |    Low            0.929    ||    High           0.690    |    Low            1.000
	              High           0.857    |    Low            0.500    ||    High           1.000    |    Low            0.651
Fold   2 :                                  Valid                          ||                          Train                      
	F1 score      BoundNet       0.733    |    SimpleNet      0.733    ||    BoundNet       1.000    |    SimpleNet      1.000
	              High           0.583    |    Low            0.824    ||    High           0.794    |    Low            

In [9]:
df_metrics = pd.DataFrame.from_dict(dict_metrics, orient='index')
mean_metrics = df_metrics.mean(axis=1)
mean_prompts = [
    f"{'BoundNet':<15}{mean_metrics[('boundnet', 'f1score', 'valid')]:.3f}{sep_model}{'SimpleNet':<15}{mean_metrics[('simplenet', 'f1score', 'valid')]:.3f}",
    f"{'High':<15}{mean_metrics[('hinet', 'f1score', 'valid')]:.3f}{sep_model}{'Low':<15}{mean_metrics[('lonet', 'f1score', 'valid')]:.3f}",
    f"{'High':<15}{mean_metrics[('hinet', 'coverage1', 'valid')]:.3f}{sep_model}{'Low':<15}{mean_metrics[('lonet', 'coverage1', 'valid')]:.3f}",
    f"{'High':<15}{mean_metrics[('hinet', 'coverage0', 'valid')]:.3f}{sep_model}{'Low':<15}{mean_metrics[('lonet', 'coverage0', 'valid')]:.3f}",
    f"{'BoundNet':<15}{mean_metrics[('boundnet', 'f1score', 'train')]:.3f}{sep_model}{'SimpleNet':<15}{mean_metrics[('simplenet', 'f1score', 'train')]:.3f}",
    f"{'High':<15}{mean_metrics[('hinet', 'f1score', 'train')]:.3f}{sep_model}{'Low':<15}{mean_metrics[('lonet', 'f1score', 'train')]:.3f}",
    f"{'High':<15}{mean_metrics[('hinet', 'coverage1', 'train')]:.3f}{sep_model}{'Low':<15}{mean_metrics[('lonet', 'coverage1', 'train')]:.3f}",
    f"{'High':<15}{mean_metrics[('hinet', 'coverage0', 'train')]:.3f}{sep_model}{'Low':<15}{mean_metrics[('lonet', 'coverage0', 'train')]:.3f}",
]
print(f"Average  :            {'Valid':^49}{sep_tv}{'Train':^49}",
          f"\tF1 score      {mean_prompts[0]}{sep_tv}{mean_prompts[4]}",
          f"\t              {mean_prompts[1]}{sep_tv}{mean_prompts[5]}",
          f"\tCoverage      {mean_prompts[2]}{sep_tv}{mean_prompts[6]}",
          f"\t              {mean_prompts[3]}{sep_tv}{mean_prompts[7]}",
          sep='\n')

Average  :                                  Valid                          ||                          Train                      
	F1 score      BoundNet       0.704    |    SimpleNet      0.728    ||    BoundNet       1.000    |    SimpleNet      1.000
	              High           0.564    |    Low            0.739    ||    High           0.791    |    Low            0.852
	Coverage      High           0.486    |    Low            0.893    ||    High           0.658    |    Low            1.000
	              High           0.829    |    Low            0.486    ||    High           1.000    |    Low            0.652
